In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Cargar el archivo
file_path = 'attachments/Conjunto_servidores_p_blicos_20240819.csv'
df = pd.read_csv(file_path, dtype={'Columna_8': str, 'Columna_9': str}, low_memory=False)

df = df[['Meses de Experiencia Público', 'Meses de Experiencia Privado',
        'Nivel Educativo', 'Tipo de Nombramiento', 'Nivel Jerarquico Empleo',
        'Dependencia Empleo Actual', 'Asignación Básica Salarial']]

# Convertir las columnas numéricas y rellenar nulos
df.loc[:, 'Meses de Experiencia Público'] = pd.to_numeric(df['Meses de Experiencia Público'], errors='coerce').fillna(0)
df.loc[:, 'Meses de Experiencia Privado'] = pd.to_numeric(df['Meses de Experiencia Privado'], errors='coerce').fillna(0)

df = df.dropna(subset=['Nivel Educativo', 'Nivel Jerarquico Empleo', 'Dependencia Empleo Actual', 'Asignación Básica Salarial'])

# Convertir la columna 'Asignación Básica Salarial' a valores numéricos y escalarla a millones
df['Asignación Básica Salarial'] = df['Asignación Básica Salarial'].replace({'$': '', ',': ''}, regex=True).astype(float)
df['Asignación Básica Salarial'] = df['Asignación Básica Salarial'] / 1_000_000

# One-Hot Encoding para las columnas categóricas
df = pd.get_dummies(df, columns=['Nivel Educativo', 'Tipo de Nombramiento', 'Nivel Jerarquico Empleo', 'Dependencia Empleo Actual'])

# Definir características (X) y etiqueta (y)
X = df.drop(columns=['Asignación Básica Salarial'])
y = df['Asignación Básica Salarial']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir todas las columnas de X_train y X_test a tipo int
X_train = X_train.astype(int)
X_test = X_test.astype(int)

# Definir el modelo de regresión lineal
model = Sequential()

# Agregar capas Dense con funciones de activación lineales
model.add(Dense(256, input_shape=(X_train.shape[1],)))  # Capa oculta
model.add(Dense(128))  # Capa oculta
model.add(Dense(64)) # Otra capa oculta
model.add(Dense(1))  # Capa de salida

# Compilar el modelo con diferentes hiperparámetros
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='mse', metrics=["mae"])

# Resumen del modelo
model.summary()

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=128, validation_split=0.2, verbose=1)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_mse = model.evaluate(X_test, y_test)


# Acceder a los valores de pérdida y mse del historial de entrenamiento
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_mse = history.history['mse']
val_mse = history.history['val_mse']

# Graficar la pérdida (loss) a lo largo de las épocas
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Pérdida de Entrenamiento')
plt.plot(val_loss, label='Pérdida de Validación')
plt.title('Pérdida durante el Entrenamiento y Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (MAE)')
plt.legend()

# Graficar el MSE a lo largo de las épocas
plt.subplot(1, 2, 2)
plt.plot(train_mse, label='MSE de Entrenamiento')
plt.plot(val_mse, label='MSE de Validación')
plt.title('MSE durante el Entrenamiento y Validación')
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend()

plt.show()

c:\Users\aleja\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │     2,232,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,274,049 (8.67 MB)

 Trainable params: 2,274,049 (8.67 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
888/888 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - loss: 10.7812 - mae: 1.7142 - val_loss: 1.7106 - val_mae: 0.8757
Epoch 2/100
888/888 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 1.7745 - mae: 0.8700 - val_loss: 1.8094 - val_mae: 0.9678
Epoch 3/100
888/888 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 1.7705 - mae: 0.8576 - val_loss: 1.6786 - val_mae: 0.8462
Epoch 4/100
888/888 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 1.7003 - mae: 0.8577 - val_loss: 1.5619 - val_mae: 0.8064
Epoch 5/100
888/888 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 1.6339 - mae: 0.8358 - val_loss: 1.8879 - val_mae: 0.9467
Epoch 6/100
888/888 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 1.6297 - mae: 0.8359 - val_loss: 1.5809 - val_mae: 0.8618
Epoch 7/100
888/888 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - loss: 1.6018 - mae: 0.8185 - val_loss: 1.5468 - val_mae: 0.7969
Epoch 8/100
888/888 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - loss: 1.5592 - mae: 0.8132 - val_loss: 2.7209 - val_mae: 1.1170
Epoch 9/100
888/888 ━━━━━━━━━━━

KeyError: 'mse'